In [25]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pickle

In [26]:
# Chrome Driver 104
DRIVER_PATH = '/Users/edwardhuang/Documents/interview/Dave And Busters/chromedriver'

# Use Chrome Beta, compatible with above Chrome Driver
options = Options()
#options.headless = True
options.binary_location = '/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta'

In [27]:
# Set up the driver
driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)
driver.implicitly_wait(30)

# Launch DnB Homepage
driver.get('https://www.daveandbusters.com')

# Launch DnB Locations Page
locationPage = WebDriverWait(driver,10).until(
    EC.presence_of_element_located(
        (By.PARTIAL_LINK_TEXT,'Location'),
    )
).click()

<ipython-input-27-79e19c123114>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)


In [28]:
# Cache all the location links v2
address1List = []
address2List = []
address3List = []
locationList = []

fullLocTable = driver.find_element(By.ID, 'dnb-locations-list')
fullLocTableExMap = fullLocTable.find_element(By.CSS_SELECTOR,'div[class="row"]')
#list, column elements
locCols = fullLocTableExMap.find_elements(By.CSS_SELECTOR,'div[class="col-md-3"]')
#list of list, state elements in column lists
states = [col.find_elements(By.CSS_SELECTOR,'div[class="location-list-item"]') for col in locCols]
#list of list, location elements in state lists
locations = [state.find_element(By.TAG_NAME,'ul').find_elements(By.CSS_SELECTOR,'li[class="dnb-location-nowopen-bullet"]') for col in states for state in col]
locationsFlat = [location for state in locations for location in state]
locationLink = [location.find_element(By.TAG_NAME,'a').get_attribute('href') for location in locationsFlat]
                

In [29]:
# Get the Address v2
for location in locationLink:
    driver.get(location)

    #Find the address
    addresses = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'div[class="location-address ng-binding'),))
    #Find the city state
    citystate = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[class="location-address-2 ng-binding"]'),))

    #Address 1
    address1List.append(addresses[0].get_attribute('innerHTML'))
    #Address 2
    address2List.append(addresses[1].get_attribute('innerHTML'))
    #City State Zip
    address3List.append(citystate.get_attribute('innerHTML'))

            

In [30]:
# Put info into a df
df = pd.DataFrame(data = {
    'Address 1': address1List,
    'Address 2': address2List,
    'City State Zip': address3List
})

# TO DO: city, state, zip
df[['City', 'State', 'Zip']] = df['City State Zip'].str.split(',', expand=True)

# Create full address
df['Full Address'] = df['Address 1'] + ' ' + df['Address 2'] + ' ' + df['City State Zip']


In [35]:
#df.to_pickle('dnb_addresses')
#df = pd.read_pickle('dnb_addresses')

In [40]:
# Google reviews v2
# I could use Action Chains to make this cleaner, but will keep as is for now.
n = 1
reviewList = []

for store in df['Full Address']:    
    driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)
    driver.implicitly_wait(30)
    driver.get('https://www.google.com')

    # cookies
    try:
        button = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.XPATH,"/html/body/div[2]/div[2]/div[3]/span/div/div/div/div[3]/div[1]/button[2]/div"))
        ).click()
    except:
        pass
    
    # Find the search bar
    searchBar = WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.NAME,"q"))
    )
    
    # Search the store
    searchBar.send_keys('Dave and Busters ' + store)
    searchBar.send_keys(Keys.ENTER)
    
    # Save the review
    try:
        review = driver.find_element(
            By.CLASS_NAME,'Aq14fc').text

        reviewList.append(review)
    except:
        reviewList.append("")
        print("Couldn't retrieve ", store)

#driver.close()

<ipython-input-40-91f293999b1b>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)


In [41]:
df['Reviews'] = reviewList
df.to_excel('dnb_stores.xlsx')
df

,Address 1,Address 2,City State Zip,City,State,Zip,Full Address,Reviews
0,2700 Riverchase Galleria,Ste 110,"Birmingham, AL, 35244",Birmingham,AL,35244,"2700 Riverchase Galleria Ste 110 Birmingham, A...",4.1
1,950 Makers Way NW,,"Huntsville, AL, 35806",Huntsville,AL,35806,"950 Makers Way NW Huntsville, AL, 35806",4.3
2,800 East Dimond Blvd.,Suite 240,"Anchorage, AK, 99515",Anchorage,AK,99515,"800 East Dimond Blvd. Suite 240 Anchorage, AK,...",4.0
3,9460 West Hanna Lane,,"Glendale, AZ, 85305",Glendale,AZ,85305,"9460 West Hanna Lane Glendale, AZ, 85305",4.2
4,21001 N. Tatum Blvd.,Suite 44-1400,"Phoenix, AZ, 85050",Phoenix,AZ,85050,"21001 N. Tatum Blvd. Suite 44-1400 Phoenix, AZ...",4.3
...,...,...,...,...,...,...,...,...
143,1101 Outlet Collection Drive SW,,"Auburn, WA, 98001",Auburn,WA,98001,"1101 Outlet Collection Drive SW Auburn, WA, 9...",3.8
144,11639 NE 4th Street,,"Bellevue, WA, 98004",Bellevue,WA,98004,"11639 NE 4th Street Bellevue, WA, 98004",3.6
145,202 Bay Park Square,,"Green Bay, WI, 54304",Green Bay,WI,54304,"202 Bay Park Square Green Bay, WI, 54304",3.8
146,414 West Towne Mall,#C,"Madison, WI, 53719",Madison,WI,53719,"414 West Towne Mall #C Madison, WI, 53719",3.7
